# COVID-19 Dashboard

### Data

[Public Health England](https://www.gov.uk/government/organisations/public-health-england) (PHE) is one of 
many Institutions worldwide running a Coronavirus [dashboard](https://coronavirus.data.gov.uk/), 
with current statistics on the pandemic.  
This dashboard accesses the COVID-19 data via the [PHE API server](https://coronavirus.data.gov.uk/developers-guide) using the Python [Software Devekopment Kit (SDK)](https://publichealthengland.github.io/coronavirus-dashboard-api-python-sdk/index.html) offered by the PHE.

In [ ]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from uk_covid19 import Cov19API

from datetime import date
from datetime import datetime

In [ ]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Load JSON files and store the raw data in some variable. Edit as appropriate:

with open("testingseries.json", "rt") as INF:
    jsondata = json.load(INF)

#filters = [
    #'areaType=overview',
#]

#structure = {
    #"date": "date",
    #"PillarOne":"newPillarOneTestsByPublishDate",
    #"PillarTwo":"newPillarTwoTestsByPublishDate",
    #"PillarThree":"newPillarThreeTestsByPublishDate",
    #"PillarFour":"newPillarFourTestsByPublishDate",
    #"Total":"newTestsByPublishDate"
#}

#api = Cov19API(filters=filters, structure=structure)

#jsondata=api.get_json()

#with open("testingseries.json", "wt") as OUTF:
    #json.dump(jsondata, OUTF)
    

In [ ]:
# Wrangling the data + defining the DataFrame:

def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(jsondata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    # Actual data is stored as a list of dictionaries under the data key, retrieveing it:
    datalist=jsondata['data']
    # Extracting all the dates and sorting them. 
    dates=[dictionary['date'] for dictionary in datalist]
    # With dates written year-first, alphabetical ordering does the trick:
    dates.sort()
    
    # Finding the earliest and latest date and converting them to the pandas type for representing dates:
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    # Creating an index as a date_range - the date analog of a range for integers, and it will include any dates that may be missing from our list:
    index=pd.date_range(startdate, enddate, freq='D') # 'D' - for day
    # Defining the DateFrame object by specifying its index and the title of its columns:
    df=pd.DataFrame(index=index, columns=['PillarOne', 'PillarTwo', 'PillarThree', 'PillarFour', 'Total']) # Constructor 
    
    # Filling the DataFrame with our data:
    for entry in datalist: # Each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['PillarOne', 'PillarTwo', 'PillarThree', 'PillarFour', 'Total']:
        # Check that nothing is there yet - just in case some dates are duplicated, 
        # maybe with data for different columns in each entry:
            if pd.isna(df.loc[date, column]): 
                # To detect missing values; isna = is None. 
                # Replacing None with 0 in our data:
                value=float(entry[column]) if entry[column]!=None else 0.0
                # Accessing a specific location in the dataframe - using df.loc
                # and inputting - index, column - in a single set of [ ]:
                df.loc[date, column]=value
            
    # Filling in any remaining "holes" due to missing dates:
    df.fillna(0.0, inplace=True)
    
    return df

# Putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
testingseriesdf=wrangle_data(jsondata) # ...df is the dataframe for plotting.

# testingseriesdf

In [ ]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback:

def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=overview',
    ]

    structure = {
        "date": "date",
        "PillarOne":"newPillarOneTestsByPublishDate",
        "PillarTwo":"newPillarTwoTestsByPublishDate",
        "PillarThree":"newPillarThreeTestsByPublishDate",
        "PillarFour":"newPillarFourTestsByPublishDate",
        "Total":"newTestsByPublishDate"
    }
    
    api = Cov19API(filters=filters, structure=structure)
    jsondata=api.get_json()

    #with open("testingseries.json", "wt") as OUTF:
        #json.dump(jsondata, OUTF)
    
    #with open("testingseries.json", "rt") as INF:
        #jsondata=json.load(INF)
    
    return jsondata 

In [ ]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button:

def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Getting fresh data from the API. If you have time, include some error handling - via except - 
    # around this call:
    
    def refresh_graph():
        """ We change the value of the widget in order to force a redraw of the graph;
        this is useful when the data have been updated. This is a bit of a gimmick; it
        needs to be customised for one of your widgets. """
        current=scale.value
        if current==scale.options[0]:
            other=scale.options[1]
        else:
            other=scale.options[0]
        scale.value=other # forces the redraw
        scale.value=current # now we can change it backdef refresh_graph():
        
    try: 
        apidata=access_api()
        # Wrangling the data and overwriting the dataframe for plotting:
        global df
        df=wrangle_data(apidata)
        # The graph won't refresh until the user interacts with the widget.
        # this function simulates the interaction, see Graph and Analysis below.
        # you can omit this step in the first instance
        refresh_graph()
        # After all is done, you can switch the icon on the button to a "check" sign
        # and optionally disable the button - it won't be needed again. You can use icons
        # "unlink" or "times" and change the button text to "Unavailable" in case the 
        # API call fails:
        apibutton.icon="check"
        apibutton.disabled=True
        
        # Show time stamp:
        today=date.today()
        d1=today.strftime("%B %d, %Y")
        now=datetime.now()
        d2=now.strftime("%H:%M:%S")
        print(f'Data last refreshed at {d2} on {d1}.', end='\r', flush=True)
    
    except:
        # apibutton.description="Error"
        # apibutton.button_style='danger'
        # apibutton.icon='exclamation-triangle'
        print('Refresh failed. Try again later.', end='\r', flush=True)
    

### Test processing in the UK

As per [Full Fact](https://fullfact.org/health/september-coronavirus-testing/) coverage:

"There are broadly [two types of test](https://www.gov.uk/government/publications/how-tests-and-testing-kits-for-coronavirus-covid-19-work/for-patients-the-public-and-professional-users-a-guide-to-covid-19-tests-and-testing-kits#how-the-tests-work):
- **Diagnostic tests** which check whether someone currently has COVID-19.
- **Antibody tests** which check whether they have had COVID-19 in the past.

These tests are sorted into four testing **“pillars”**. 

**Pillar 1** tests are diagnostic tests done in Public Health England labs and hospitals for health and care workers and patients who are seriously ill. Specifically, they test for the prescence of the virus's genetic information using
PCR.

**Pillar 2** tests are also diagnostic tests, but done by commercial partners for the wider population. These tests are done at regional test sites, mobile testing units, satellite test centres and via home tests. 

More details on the differences can be found [here](https://www.gov.uk/guidance/coronavirus-covid-19-getting-tested).

**Pillar 3** covers the antibody tests.

**Pillar 4** comprises a mix of antibody and diagnostic tests for the purposes of national surveillance and for researchers to understand more about the spread of the disease. However, positive cases identified through these studies are also reallocated to Pillar 1 and Pillar 2."



In [ ]:
# Graph + interactive widgets:

series=wdg.SelectMultiple(
    options=['PillarOne', 'PillarTwo', 'PillarThree', 'PillarFour', 'Total'],
    value=['PillarOne', 'PillarTwo', 'PillarThree', 'PillarFour'],
    rows=5,
    description='Tests:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

def series_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0: # Error check
        testingseriesdf[list(gcols)].plot(logy=logscale, title='Number of COVID-19 tests processed in the UK')
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
apibutton=wdg.Button(
    description="Refresh",
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to download current Public Health England data!",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='refresh' # e.g. exclamation-triangle
)

# Remember to register your button callback function with the button:
apibutton.on_click(api_button_callback) # The name of your function inside these brackets.
        
# Keep calling series_graph(gcols=value_of_series, gscale=value_of_scale); capture output in variable graph   
graph=wdg.interactive_output(series_graph, {'gcols': series, 'gscale': scale})
ctrls=wdg.VBox([series, scale]) # stacked
form=wdg.HBox([graph, ctrls]) # side-by-side

display(form, apibutton)

This graph illustrates how the test processing in the UK changed over the course of the year, with available data starting in April.  
You can view the data plotted in linear or logarithmic scale, and compare the different test types' processing.  
The refresh button fetches the new data from the PHE API server, which updates on the rolling 7-day basis.

**(c) 2020 Phuong Mai Nguyen** ([p.nguyen@se20.qmul.ac.uk](mailto:p.nguyen@se20.qmul.ac.uk)), all rights reserved.

*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*